# Ranch Demo Notebook

This notebook has examples of how to make Ranch calls to run through the complete network building pipeline to develop standard roadway and transit network from scratch

In [1]:
import os
import pickle
import pandas as pd
import geopandas as gpd
from pyproj import CRS

import ranch
from ranch import sharedstreets
from ranch import Roadway
from ranch import Transit
from ranch import Parameters
from ranch.utils import link_df_to_geojson, point_df_to_geojson
from ranch.logger import RanchLogger

In [2]:
%load_ext autoreload
%autoreload 2

# Remote I/O

User to update, does not necessarily be the ranch directory

In [3]:
# project directory
root_dir = os.path.join("D:/sanjoaquin")

external_dir = os.path.join(root_dir, "data", "external")
interim_dir = os.path.join(root_dir, "data", "interim")

# software directory
ranch_dir = os.path.join("D:/github/Ranch")

In [4]:
parameters = Parameters(ranch_base_dir = ranch_dir)

2021-12-14 15:22:49, INFO: Lasso base directory set as: D:/github/Ranch


## Step 1 - SharedStreets Extraction

Extracts sharedstreets representation of OSM links.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from SharedStreets API.

In [5]:
# the polygon file for the area

input_polygon_file = os.path.join(
    external_dir,
    "sharedstreets_extract", 
    "sanjoaquin.shp"
)

In [6]:
# the folder where SharedStreets extractions live

shst_extract_dir = os.path.join(external_dir, "sharedstreets_extract")

In [18]:
# the SharedStreets extraction file is saved to the 'output_dir' argument

ranch.run_shst_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = shst_extract_dir
)

2021-12-14 16:45:29, INFO: Exporting boundry file D:/sanjoaquin\data\external\sharedstreets_extract\boundary.0.geojson
2021-12-14 16:45:29, INFO: extracting for polygon 0


## Step 2 - OSMNX Extraction

Extracts complete OSM attributes using OSMNX.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from OSMNX.

In [12]:
# the folder where OSMNX extractions live

osm_extract_dir = os.path.join(external_dir, "osmnx_extract")

In [13]:
# the OSMNX extraction file is saved to the 'output_dir' argument

ranch.run_osmnx_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = os.path.join(external_dir, "osmnx_extract")
)

## Step 3 - Consolidate SharedStreets and OSMNX exractions

consolidates raw roadway data and create initial roadway networks

In [15]:
# build and returns roaday network object from extraction files

roadway_network = Roadway.create_roadway_network_from_extracts(
    shst_extract_dir = shst_extract_dir,
    osm_extract_dir = osm_extract_dir,
    parameters = parameters,
)

2021-12-14 16:20:22, INFO: Reading sharedstreets data
2021-12-14 16:20:22, INFO: ----------start reading shst extraction data-------------
2021-12-14 16:20:22, INFO: reading shst extraction data : D:/sanjoaquin\data\external\sharedstreets_extract\extract.boundary.0.out.geojson
2021-12-14 16:20:55, INFO: reading shst extraction data : D:/sanjoaquin\data\external\sharedstreets_extract\out.out.geojson
2021-12-14 16:20:55, INFO: ----------finished reading shst extraction data-------------
2021-12-14 16:20:55, INFO: Removing duplicates in shst extraction data
2021-12-14 16:20:55, INFO: ...before removing duplicates, shst extraction has 99961 geometries.
2021-12-14 16:20:55, INFO: ...after removing duplicates, shst extraction has 99961 geometries.
2021-12-14 16:20:55, INFO: Reading osmnx data
2021-12-14 16:22:38, INFO: Extracting corresponding osm ways for every shst geometry
2021-12-14 16:24:35, INFO: shst extraction has 99961 geometries
2021-12-14 16:24:35, INFO: shst extraction has 107067

C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\ranch\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


2021-12-14 16:25:43, INFO: Start creating shst nodes


C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\ranch\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\ranch\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


2021-12-14 16:25:57, INFO: Lasso base directory set as: D:/github/Ranch


In [16]:
RanchLogger.info("Initial network has {} links".format(roadway_network.links_df.shape[0]))
RanchLogger.info("Initial network has {} nodes".format(roadway_network.nodes_df.shape[0]))
RanchLogger.info("Initial network has {} shapes".format(roadway_network.shapes_df.shape[0]))

2021-12-14 16:42:31, INFO: Initial network has 192247 links
2021-12-14 16:42:31, INFO: Initial network has 73049 nodes
2021-12-14 16:42:31, INFO: Initial network has 99961 shapes


### Optional: If user wants to write out the roadway network in standard format, they can do the following, otherwise no need to write out

In [ ]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step3_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step3_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(root_dir,"data", "interim","step3_links.geojson"), "w") as f:
    json.dump(link_geojson, f)


## Step 4 - Third-Party Roadway Data Conflation

Conflates e.g. county network with roadway network using SharedStreets.
The input for this step is the 3rd-party geodatabases to conflate.
The output of this step is the SharedStreets conflation outputs in geojson.

In [19]:
# define the input 3rd-party file, either shapefile or geojson

input_network_file = os.path.join(external_dir, "sjmodel", "Network", "2015","TCM_MASTER_2A22_102717.shp")

In [20]:
# if the CRS is undefined in the input network file, user needs to specify:

input_crs = CRS("ESRI:102643")

In [21]:
# the output is saved to the 'output_dir' argument
# user can specify the match option (reference here) 
# by 'custom_match_option'
# if not specified, use the default
# match option would vary by the input network

ranch.run_shst_match(
    input_network_file = input_network_file,
    input_crs = input_crs,
    output_dir = os.path.join(external_dir, "sjmodel", "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

2021-12-14 16:47:54, INFO: input network D:/sanjoaquin\data\external\sjmodel\Network\2015\TCM_MASTER_2A22_102717.shp has crs : None
2021-12-14 16:47:56, INFO: Input network for shst match does not have unique IDs, generating unique IDs
2021-12-14 16:47:56, INFO: Generated 49910 unique IDs for 49910 links in the input network
2021-12-14 16:47:56, INFO: Exporting shst match input - ID-ed geometry file D:/sanjoaquin\data\external\sjmodel\shst_match\TCM_MASTER_2A22_102717.geojson
2021-12-14 16:48:00, INFO: Exporting ID-ed network file D:/sanjoaquin\data\external\sjmodel\shst_match\TCM_MASTER_2A22_102717.full.geojson


## Step 5 - Tidy Up Roadway

Identifies drive dead-ends, cul-de-secs, numbering links and nodes, etc.
The input for this step is the polygon file with subregion identifier, e.g. county.
This step labels each link and node with the county name, and assign model IDs for links and nodes.

In [ ]:
# this is the input county polygon file with county names

county_boundary_file = os.path.join(
    external_dir,
    "cb_2018_us_county_500k", 
    "san_joaquin.shp"
)

# specify the column to look for county name

county_variable_name = 'NAME'

In [ ]:
roadway_network.tidy_roadway(
    county_boundary_file = county_boundary_file,
    county_variable_name = county_variable_name
)

## Step 6 - Build Transit Network from GTFS

Build standard transit network from GTFS

In [ ]:
gtfs_dir = os.path.join(external_dir, "gtfs", "2015")

In [ ]:
# read gtfs into transit object

transit_network = Transit.load_all_gtfs_feeds(
    path = gtfs_dir,
    roadway_network= roadway_network,
    parameters=parameters
)

In [ ]:
# main activities - routing buses, creating rails

transit_network.build_standard_transit_network()

In [ ]:
# write out

transit_network.write_standard_transit(
    path = interim_dir
)

## Step 7 - Build Centroid Connectors

builds centroid connectors from TAZ, MAZ

In [ ]:
taz_polygon_file = os.path.join(
    external_dir,
    "taz",
    "SJ_TAZ_Aug2011.shp"
)

In [ ]:
roadway_network.build_centroid_connectors(
    build_taz_drive = True,
    build_taz_active_modes = True,
    input_taz_polygon_file = taz_polygon_file
)

## Step 8 - Write out Standard Format

In [ ]:
roadway_network.standard_format(
    county_boundary_file = os.path.join(root_dir,"data", "external","cb_2018_us_county_500k", "san_joaquin.shp"),
    county_variable_name = 'NAME'
)

In [ ]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step8_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step8_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step8_links.geojson"), "w") as f:
    json.dump(link_geojson, f)
